In [95]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [96]:
# Load the dataset using pandas 
df = pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [97]:
# Drop Cabin because its has too many missing values
df.drop('Cabin', axis = 1, inplace = True)
df.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
434,435,0,1,"Silvey, Mr. William Baird",male,50.0,1,0,13507,55.9000,S
404,405,0,3,"Oreskovic, Miss. Marija",female,20.0,0,0,315096,8.6625,S


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [99]:
# Lets extract those columns on which I'm going to work
df_extract = df.iloc[:, [1,4,5,-1]]
df_extract.head(3)

,Survived,Sex,Age,Embarked
0,0,male,22.0,S
1,1,female,38.0,C
2,1,female,26.0,S


In [100]:
X = df[['Sex', 'Age', 'Embarked']]
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.sample(2)

,Sex,Age,Embarked
57,male,28.5,C
201,male,NaN,S


In [101]:
# One-Hot encoding Sex
ohe = OneHotEncoder(drop = 'first', sparse_output = False, handle_unknown = 'ignore')

X_train_sex = ohe.fit_transform(X_train[['Sex']])
X_test_sex = ohe.transform(X_test[['Sex']])

In [102]:
X_train['Age'].isnull().sum()

140

In [103]:
# Imputing missing values in Age(numerical)
si = SimpleImputer(strategy = 'mean')

X_train_age = si.fit_transform(X_train[['Age']])
X_test_age = si.transform(X_test[['Age']])


In [104]:
# Embarked column has missing values and also needs encoding
# 1st impute missing values using most frequent strategy

si_emb = SimpleImputer(strategy = 'most_frequent')

X_train_embarked = si_emb.fit_transform(X_train[['Embarked']])
X_test_embarked = si_emb.transform(X_test[['Embarked']])

In [105]:
# 2nd encoding Embarked column
ohe_emb = OneHotEncoder(drop = 'first', sparse_output = False, handle_unknown = 'ignore')

X_train_embarked_enc = ohe_emb.fit_transform(X_train_embarked)
X_test_embarked_enc = ohe_emb.transform(X_test_embarked)

In [106]:
# Now concatinate all the preprosessing steps
X_train_transformed = np.concatenate((X_train_sex, X_train_age, X_train_embarked_enc), axis = 1)
X_test_transformed = np.concatenate((X_test_sex, X_test_age, X_test_embarked_enc), axis = 1)

In [107]:
X_train_transformed

array([[ 1. , 45.5,  0. ,  1. ],
       [ 1. , 23. ,  0. ,  1. ],
       [ 1. , 32. ,  0. ,  1. ],
       ...,
       [ 1. , 41. ,  0. ,  1. ],
       [ 0. , 14. ,  0. ,  1. ],
       [ 1. , 21. ,  0. ,  1. ]])